# Ejercicio: análisis semántico con word2vec

<img src="img/word2vec.png" style="width:600px;height:400px;">

En este ejercicio vamos a utilizar word2vec para estudiar algunas relaciones semánticas entre palabras. Veremos cómo con esta técnica podemos resolver fácilmente los típicos problemas de encontrar palabras extrañas en un conjunto dado.

## Instrucciones

A lo largo de este cuaderno encontrarás celdas vacías que tendrás que rellenar con tu propio código. Sigue las instrucciones del cuaderno y presta especial atención a los siguientes iconos:

<table>
<tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">Deberás responder a la pregunta indicada con el código o contestación que escribas en la celda inferior.</td></tr>
 <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">Esto es una pista u observación que te puede ayudar a resolver la práctica.</td></tr>
 <tr><td width="80"><img src="img/pro.png" style="width:auto;height:auto"></td><td style="text-align:left">Este es un ejercicio avanzado y voluntario que puedes realizar si quieres profundar más sobre el tema. Te animamos a intentarlo para aprender más ¡Ánimo!</td></tr>
</table>

Para evitar problemas de compatibilidad y de paquetes no instalados, se recomienda ejecutar este notebook bajo uno de los [entornos recomendados de Text Mining](https://github.com/albarji/teaching-environments/tree/master/textmining).

Adicionalmente si necesitas consultar la ayuda de cualquier función python puedes colocar el cursor de escritura sobre el nombre de la misma y pulsar Mayúsculas+Shift para que aparezca un recuadro con sus detalles. Ten en cuenta que esto únicamente funciona en las celdas de código.

¡Adelante!

## Carga y preparación de datos

Entrenar un modelo de word2vec es una tarea muy costosa computacionalmente, que además requiere de corpus de texto muy grandes, del orden de miles de millones de palabras. Afortunadamente existen modelos word2vec pre-entrenados que están disponibles de forma pública y con los que podemos trabajar para hacer nuestros análisis semánticos. Uno de ellos es el modelo [GoogleNews-vectors-negative300](https://code.google.com/archive/p/word2vec/), entrenado con 100.000 millones de palabras y que se encuentra disponible para descarga en la siguiente dirección: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing.

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
      Descarga manualmente el modelo <b>GoogleNews-vectors-negative300</b> en tu máquina, y a continuación crea una variable <i>modelfile</i> con la ruta del fichero descomprido del modelo.
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

## Similitud semántica

Un modelo pre-entrenado word2vec no es más que un diccionario en el que para cada palabra tenemos el vector que la representa. Podemos cargar en memoria este diccionario utilizando el paquete de análisis de texto **gensim**:

In [ ]:
import gensim
embeddings = gensim.models.KeyedVectors.load_word2vec_format(modelfile, binary=True)

Es conveniente aplicar la siguiente operación para guardar en memoria únicamente la información relativa a los vectores. Esto hace que no podamos reentrenar la representación vectorial de las palabras, pero para el objetivo de esta práctica no es necesario tal cosa.

In [ ]:
embeddings.init_sims(replace=True)

Podemos comprobar ahora la representación vectorial de diferentes palabras, por ejemplo:

In [ ]:
embeddings["queen"]

Aunque la representación vectorial de una palabra resulta oscura de interpretar, se ha comprobado que sigue una lógica semántica y sintáctica. Esto permite hacer aritmética con estos vectores y obtener resultados que son coherentes con lo que cabría esperar. Por ejemplo, si denotamos como $v(word)$ la representación vectorial de una cierta palabra *word* podemos encontrar casos como: 

$$v(king) - v(man) + v(woman) \simeq v(queen)$$

Podemos comprobar que eso es cierto utilizando la función **most_similar** del objeto que contiene los embeddings. Esta función recibe dos listas de palabras, a contribuir de forma positiva o negativa a la operación aritmética, y devuelve las palabras cuya representación vectorial sea más cercana al vector resultado de la operación, ordenadas por similitud:

In [ ]:
embeddings.most_similar(positive=['king', 'woman'], negative=['man'])

¡Funciona! Podemos ver más ejemplos, como los siguientes:

In [ ]:
embeddings.most_similar(positive=['Bush', 'Russia'], negative=['USA'])

In [ ]:
embeddings.most_similar(positive=['Madrid', 'France'], negative=['Spain'])

In [ ]:
embeddings.most_similar(positive=['goodbye', 'spanish'], negative=['english'])

In [ ]:
embeddings.most_similar(positive=['boat', 'air'], negative=['ocean'])

In [ ]:
embeddings.most_similar(positive=['paella', 'Italy'], negative=['Spain'])

In [ ]:
embeddings.most_similar(positive=['Harry_Potter', 'evil'], negative=['good'])

In [ ]:
embeddings.most_similar(positive=['artificial_intelligence', 'evil'], negative=['good'])

Incluso pueden capturarse relaciones de morfología entre palabras, como las siguientes:

In [ ]:
embeddings.most_similar(positive=['sister', 'he'], negative=['she'])

In [ ]:
embeddings.most_similar(positive=['cat', 'many'], negative=['one'])

In [ ]:
embeddings.most_similar(positive=['eat', 'past'], negative=['present'])

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
    ¡Busca tú mismo algún ejemplo interesante!
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

## Odd-one out

Una utilidad interesante de las distancias semánticas entre palabras es de resolver los típicos pasatiempos en los que se debe identificar la palabra que no encaja dentro de un grupo dado. Por ejemplo:

In [ ]:
group = ["Obama", "Merkel", "Putin", "truck"]

Está claro que *truck* es la palabra intrusa en esta lista de presidentes del gobierno. Pero esto es algo que sabemos por nuestro amplio conocimiento del mundo y del lenguaje, y para un programa informático no es nada trivial llegar a esta conclusión. Sin embargo gracias a las representaciones semánticas en forma de vector que nos da word2vec podemos hacerlo.

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
      Implementa una función <b>oddoneout</b> que recibe una lista de palabras y un modelo word2vec, y realice los siguientes pasos:
    <ol>
     <li> Obtener la representación vectorial de cada palabra recibida. Ignora las palabras para las que el modelo no contemple una representación vectorial.</li>
     <li> Calcula el vector medio de todas las palabras recibidas.</li>
     <li> Calcula la distancia de ese vector medio al vector representativo de cada palabra.</li>
     <li> Devuelve la palabra con mayor distancia.</li>
    </ol>
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

Vamos a comprobar ahora con el ejemplo de antes si la implementación ha funcionado:

In [ ]:
oddoneout(group, embeddings)

<table>
 <tr>
  <td width="80"><img src="img/pro.png" style="width:auto;height:auto"></td><td style="text-align:left">
    ¿Ha funcionado el ejemplo anterior? ¿Puedes pensar en otros ejemplos en los que el algoritmo también fucione? ¿Hay algún caso en el que falle?
     En gensim cualquier objeto de modelo word2vec dispone de la función <b>doesnt_match</b> que realiza la misma función que el algoritmo que has implementado, pero con un cálculo de distancias más adecuado a la representación vectorial. En general esta aproximación debería ser mejor que la que has implementado. ¿Encuentras algún caso en el que sea así?
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE